# 질문할 것

- 멤버십 체크 부분
    - 제대로 동작하지 않음.
    - 검색 결과 페이지가 3페이지(총 27건)까지 밖에 없어서 4번째 response에서 끝나야 되는데 계속 반복됨.
    
    
- lines = ''' ~ '''
    - 코드가 뭔가 안 이쁨. (들여쓰기가 안 되서 가독성이 떨어지는 것 같은..)
    - 그렇지만 들여쓰기하면 가운데 공백문자가 들어가서 내용이 바뀌게 됨.
    - 안 이뻐도 그냥 저대로 놔두어야 하나?
    - (들여쓰기해서 lstrip() 쓰면 똑같이 동작하긴 하는데, Form Data를 건드리지 않고 해결할 수 있을까?)

In [3]:
from collections import OrderedDict
from itertools import count
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup


def make_scraped_data(keyword, max_page=None): 
    scraped_data = {}
    
#     for page in count(1):
    for page in range(1, 10):
        lines = '''listPage:{}
listCount:10
keyword:{}
orderByOption:desc
fromDate:
toDate:
provider_code:
provider_name:
category_code:
category_name:
larm_incident_category_path:
larm_incident_category_nm:
search_field:_search
search_method:0
realKeyword:
keywordType:N
byline:'''.format(page, keyword).splitlines()

        data = {}
        for line in lines:
            key, value = line.split(':', 1)
            if value == 'null':
                value = None
            data[key] = value

        result_url = 'http://211.43.216.33/search/totalSearchList.do'
        r = requests.post(result_url, data=data)
        print(str(page) + ' page', r)
        soup = BeautifulSoup(r.text, 'html.parser')

        article_list = soup.select('.list div')

        for article in article_list:
            items = OrderedDict()
            items['title'] = article.select_one('dt a').text.strip()
            items['link'] = urljoin(result_url, article.select_one('dt a')['href'])
            items['discription'] = article.select_one('dd a').text

            made = article.select_one('.txt_blue11').text.strip().split()

            items['newspaper'] = made[0]
            items['created_date'] = made[2]
            try:
                items['writer'] = made[4]
            except IndexError:
                items['writer'] = None

            # 멤버십 체크
            if items['link'] in scraped_data.keys():
                return scraped_data
            scraped_data[items['link']] = items
        
        # max_page
        if max_page:
            if page >= max_page:
                break

    return scraped_data

In [4]:
result = make_scraped_data(keyword='후덜덜')  # 4페이지까지만 나오고 끝나야하는데 멤버십 체크가 안 먹음. why?

1 page <Response [200]>
2 page <Response [200]>
3 page <Response [200]>
4 page <Response [200]>
5 page <Response [200]>
6 page <Response [200]>
7 page <Response [200]>
8 page <Response [200]>
9 page <Response [200]>


In [5]:
len(result)

27

In [6]:
result

{'http://211.43.216.33/search/totalSearchView.do?news_id=01100611.20170708033918001': OrderedDict([('title',
               '후덜덜'),
              ('link',
               'http://211.43.216.33/search/totalSearchView.do?news_id=01100611.20170708033918001'),
              ('discription',
               '7일 경기 광명 광명동굴에 설치된 이색 공포체험관 ‘좀비 캐슬’ 사전 체험 행사를 찾은 체험객들이 괴물 복장을 한 연기자들을 보고 놀라고 있다. 좀비 캐슬은 8일부터 정식 운영에 들어간다. 연합뉴스'),
              ('newspaper', '서울신문'),
              ('created_date', '2017.07.08'),
              ('writer', None)]),
 'http://211.43.216.33/search/totalSearchView.do?news_id=01100701.2013112810012829656': OrderedDict([('title',
               '클라라, 무보정 직찍…재킷 안 몸매 후덜덜'),
              ('link',
               'http://211.43.216.33/search/totalSearchView.do?news_id=01100701.2013112810012829656'),
              ('discription',
               '배우 클라라의 화보 촬영 비하인드컷이 공개됐다.\xa0    명품 몸매와 섹시한 외모로 주목받고 있는 클라라는 스포츠 브랜드 스케쳐스 화보에서 후드 티셔츠와 핫팬츠, 다운재킷으로 이미지 변신에 성공했다.\xa0    클라라는 공개된 사진에서 숨겨둔 반전

In [5]:
for key, value_dict in result.items():
    for key, value in value_dict.items():
        print(key.title() + ': ' + str(value))
    print('\n')

Title: KEB하나은행-SKT 합작 앱 ‘핀크(finnq)’ 출시에 은행원들 ..
Link: http://211.43.216.33/search/totalSearchView.do?news_id=02100101.20170906103044001
Discription: 하나금융그룹이 국내 1위 이통사 SK텔레콤과 손잡고 새로운 플랫폼 앱 핀크(Finnq)을 내놨는데. 핀크는 신용카드 거래 내역을 기반으로 한 지출(소비)관리 서비스 앱. 하나은행 계좌 개설 외에 예·적금 등 여러 금융상품에 가입할 수 있고 타 은행 상품 가입, 비교도 가능. 특히 인터넷전문은행을 경쟁 상..
Newspaper: 매일경제
Created_Date: 2017.09.06
Writer: 박수호


Title: 후덜덜
Link: http://211.43.216.33/search/totalSearchView.do?news_id=01100611.20170708033918001
Discription: 7일 경기 광명 광명동굴에 설치된 이색 공포체험관 ‘좀비 캐슬’ 사전 체험 행사를 찾은 체험객들이 괴물 복장을 한 연기자들을 보고 놀라고 있다. 좀비 캐슬은 8일부터 정식 운영에 들어간다. 연합뉴스
Newspaper: 서울신문
Created_Date: 2017.07.08
Writer: None


Title: ‘싸이 흠뻑쇼’, "겨터파크 준비하고 오세요~"
Link: http://211.43.216.33/search/totalSearchView.do?news_id=02100101.20170620134519001
Discription: [매일경제 스타투데이 이은솔 인턴기자] 가수 싸이가 5년만에 여름콘서트 ‘싸이 흠뻑쇼’를 개최하는 가운데, 지난 ‘싸이 흠뻑쇼’ 콘서트 후기가 눈길을 끈다. 20일 정오 ‘싸이 흠뻑쇼-서머 스웨그'(SUMMER SWAG)’는 인터파크 티켓을 통해 예매가 시작됐다. ..“밤 샐 때까지 공연합니다..후덜덜”, 
Newspaper: 매일경제
Created_Date: 2017.0

## 참고) lstrip() 써서 수정한 코드 : 똑같이 동작하긴 함.

In [6]:
from collections import OrderedDict
from itertools import count
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup


def make_scraped_data(keyword, max_page=None): 
    scraped_data = {}
    
#     for page in count(1):
    for page in range(1, 10):
        lines = '''listPage:{}
                   listCount:10
                   keyword:{}
                   orderByOption:desc
                   fromDate:
                   toDate:
                   provider_code:
                   provider_name:
                   category_code:
                   category_name:
                   larm_incident_category_path:
                   larm_incident_category_nm:
                   search_field:_search
                   search_method:0
                   realKeyword:
                   keywordType:N
                   byline:'''.format(page, keyword).splitlines()

        data = {}
        for line in lines:
            key, value = line.split(':', 1)
#             print(key.lstrip(), value)
            if value == 'null':
                value = None
            data[key.lstrip()] = value

        result_url = 'http://211.43.216.33/search/totalSearchList.do'
        r = requests.post(result_url, data=data)
        print(str(page) + ' page', r)
        soup = BeautifulSoup(r.text, 'html.parser')

        article_list = soup.select('.list div')

        for article in article_list:
            items = OrderedDict()
            items['title'] = article.select_one('dt a').text.strip()
            items['link'] = urljoin(result_url, article.select_one('dt a')['href'])
            items['discription'] = article.select_one('dd a').text

            made = article.select_one('.txt_blue11').text.strip().split()

            items['newspaper'] = made[0]
            items['created_date'] = made[2]
            try:
                items['writer'] = made[4]
            except IndexError:
                items['writer'] = None

            # 멤버십 체크
            if items['link'] in scraped_data:
                return scraped_data
            scraped_data[items['link']] = items
        
        # max_page
        if max_page:
            if page >= max_page:
                break

    return scraped_data

In [7]:
result = make_scraped_data(keyword='후덜덜')

1 page <Response [200]>
2 page <Response [200]>
3 page <Response [200]>
4 page <Response [200]>
5 page <Response [200]>
6 page <Response [200]>
7 page <Response [200]>
8 page <Response [200]>
9 page <Response [200]>


In [8]:
len(result)

27